<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2021_0420Onomatopea_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab では以下の 2 行の行頭の # を削除してから実行してください
!pip install mecab-python3
!pip install unidic-lite
# Colab では以下の 2 行の行頭の # を削除してから実行してください
!pip install jaconv
!pip install japanize_matplotlib
#!pwd

In [ ]:
import requests

# word2vec の訓練済モデルを入手
urls =['http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz',
       'http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz',
       'http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz',
       'http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz']

url = urls[0]
w2v_fname = url.split('/')[-1]
r = requests.get(url)
with open(w2v_fname, 'wb') as f:
    total_length = int(r.headers.get('content-length'))
    print('Downloading {0} - {1} bytes'.format(w2v_fname, (total_length)))
    f.write(r.content)

w2v_base = '.'
w2v_file = os.path.join(w2v_base, w2v_fname)
w2v = KeyedVectors.load_word2vec_format(w2v_fname, 
                                        encoding='utf-8', 
                                        unicode_errors='replace',
                                        binary=True) 



In [ ]:
import os
import pandas as pd
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータ
#ひとつ下の '日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードする
from google.colab import files
uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする

ccap_base = '.'
#onomatopea_excel = '日本語オノマトペ辞典4500より.xlsx'
onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(ccap_base, onomatopea_excel), sheet_name='2761語')

In [ ]:
import sys
import numpy as np

np.set_printoptions(precision=2)  # numpy の表示桁数設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})
#torch.set_printoptions(precision=3)

import json
import matplotlib.pyplot as plt
%matplotlib inline

# word2vec データ処理のため gensim を使う
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from scipy import stats

#import tqdm
import termcolor
import jaconv  # ひらがなカタカナ変換用 `pip install jaconv` してください
import japanize_matplotlib  # matplotlib の日本語表示

In [ ]:
onomatopea = list(set(sorted(onmtp2761['オノマトペ'])))
print('# オノマトペのうち，word2vec に登録があるかどうかを調査')
kana_onmtp, kata_onmtp = [], []
count = 0
for word in onomatopea:
    count += 1
    if word in w2v.vocab:
        kana_onmtp.append(word)

    kata_w = jaconv.hira2kata(word)
    if kata_w in w2v.vocab:
        kata_onmtp.append(kata_w)
        
Vono = kana_onmtp + kata_onmtp

print(f'Word2vec (wikipedida_ja を使って訓練) に存在するオノマトペ数: {len(Vono)}，全オノマトペ項目(小野オノマトペ辞典4500語) {len(onomatopea)} 語')
print('総数がオノマトペデータより多いのは，平仮名表記とカタカナ表記と両者で wikipedia に登録があった場合に重複してカウントしているからです。')
print(f'カタカナ オノマトペ総数: {len(kata_onmtp)}')
print(f'ひらがな オノマトペ総数: {len(kana_onmtp)}')

In [ ]:
def P_orth(X):
    """Return Orthogonal Projection and its complement matrices of X
    直交射影行列とその補空間への射影行列を返す。
    X を (n行 m列)としたとき， (m,m) 型の逆行列を算出。(n,n)型ではないことに注意
    
    引数
    x: np.array (n,m)
        入力行列
    戻り値
    P: np.array (m,m)
        射影行列
    Q: np.arrary(m,m)
        直交補空間への射影行列
    """
    XT = X.T
    X_XT = np.dot(X, XT)
    iXXT = np.linalg.inv(X_XT)
    XT_iXXT = np.dot(XT,iXXT) # np.dot(XT, iXXT)
    XT_iXXT_X = np.dot(XT_iXXT, X)
    P = XT_iXXT_X
    I = np.eye((P.shape[0]))
    Q = I - P
    return P, Q

def w2vMat(w2v=w2v, wordlist=['イヌ','ネコ', 'トラ', 'ライオン']):
    """len(wordlist)行，word2vec 次元数 列を持つ行列 を返す"""

    if w2v == None:
        assert('Set a `gensim.models.keyedvectors.Word2VecKeyedVectors` as an w2v argument')
        
    # 行列の確保
    X = np.zeros((len(set(wordlist)), w2v.vector_size), dtype=np.float)
        
    # 各行に word2vec ベクトルをコピー
    for i, w in enumerate(wordlist):
        X[i] = np.copy(w2v[w])
            
    return X


Mono = w2vMat(wordlist=Vono)
print(f'#全オノマトペ項目を用いた単語埋め込みベクトル行からなる行列のサイズ:{Mono.shape}')
P_ono, Q_ono = P_orth(Mono)
print(P_ono.shape, Q_ono.shape)


In [ ]:
#カタカナ全文字を `kata_chars` へ保存
kata_chars = [c for c in 'ァアィイゥウェエォオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモャヤュユョヨラリルレロヮワヰヱヲンヴヵヶ']

vocabs = set(sorted(list(w2v.vocab)[1:]))  # word2vec に登録されている全単語を保存
kata_words = []  # word2vec in wikipedia.ja に登録されている全カタカナリストを作成
for w in Vono:
    # 全単語リスト `vocabs` からカタカナだけで構成されている単語を kata_words に登録
    kata_flag = True
    for c in w:
        if c not in kata_chars:
            kata_flag = False
            break
    if kata_flag == True:
        kata_words.append(w)

#結果の確認
print('カタカナ単語総数:{0}, 総語彙数:{1}, 比率(%):{2:.3f}'.format(len(kata_words), 
                                                     len(vocabs), 
                                                     len(kata_words)/len(vocabs) * 100))

In [ ]:
n = 0
onmtp_w2v_kata = []  #カタカナ表記のオノマトペリスト
onmtp_w2v_hira = []  #ひらがな表記のオノマトペリスト

#全カタカナ語についてオノマトペデータベースに登録のある単語であればリストに追加
for w in kata_words:
    if w in Vono:
        # カタカナ単語がオノマトペ辞典に載っている単語であればリストに追加
        onmtp_w2v_kata.append(w)

    w = jaconv.kata2hira(w)  # ひらがなに変換
    if w in Vono:
        # ひらがなに変換した単語がオノマトペ辞典に載っている単語であればリストに追加
        onmtp_w2v_hira.append(w)

print(f'# wikipeida.ja に登録されているカタカナ単語 {len(kata_words)} 語のうち')
print(f'# オノマトペ辞典に載っている単語数:{len(onmtp_w2v_kata)}, ひらがな変換するとオノマトペ辞典に載っている単語数 {len(onmtp_w2v_hira)}')

In [ ]:
print('#上で調べたword2vecにエントリの存在するオノマトペ全単語の品詞を MeCab を使って調べる')
import MeCab

#m = MeCab.Tagger() #形態素解析用オブジェクトの宣言

kata_pos_wiki = {}
print('#日本語ウィキペディア全カタカナ単語の場合:')
for w in kata_words:
    pos = str(m.parse(w).strip().split(',')[1:3])
    if pos in kata_pos_wiki:
        kata_pos_wiki[pos] += 1
    else:
        kata_pos_wiki[pos] = 1

print(json.dumps(kata_pos_wiki, ensure_ascii=False, indent=4))

print('#そのうちオノマトペ辞典に登録されている単語の場合:')
onmtp_set = set(onmtp_w2v_kata + onmtp_w2v_hira)
onmtp_pos = {}
for w in onmtp_set:
    #pos = str(m.parse(w).strip().split(',')[1:3])
    w_ = MeCab.Tagger().parse(w).strip().split(',')
    pos = w_[1] + w_[2]
    if pos not in onoma_pos:
        onmtp_pos[pos] = list()
    onmtp_pos[pos].append(w)

#print(json.dumps(onoma_pos, ensure_ascii=False, indent=4))
print(list(onmtp_pos))

In [ ]:
MeCab.Tagger().parse('りんご').strip().split('\t')

In [ ]:
#視覚化のためのライブラリを読み込む
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns

np.set_printoptions(suppress=False, formatter={'float': '{:7.5f}'.format})

onmtp_list =list(onmtp_set)
Onmtp_w2v = w2vMat(wordlist=onmtp_list)

Onmtp_w2v_norm = np.array([x/np.linalg.norm(x) for x in Onmtp_w2v])
R_onmtp = Onmtp_w2v_norm.dot(Onmtp_w2v_norm.T)
print(R_onmtp.shape)

R_onmtp_df = pd.DataFrame(data=R_onmtp, index=onmtp_list)
fig, ax = plt.subplots(figsize=(12,10))         # Sample figsize in inches
sns.heatmap(R_onmtp_df, ax=ax)

In [ ]:
def ax_scatter_gram(ax, p1, p2, wordlist, title=None, fontsize=10, color='cyan', x_label="第1軸", y_label="第2軸"):
    ax.scatter(p1, p2, s=60, color=color)
    for i, label in enumerate(wordlist):
        ax.annotate(label, (p1[i], p2[i]),fontsize=fontsize)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title,fontsize=fontsize*1.2)

def plot_pca(ax, R, wordlist, title=""):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(R)
    pca1, pca2 = pca_result[:,0], pca_result[:,1] 
    print('\tExplained variation per principal component: {}'.format(pca.explained_variance_ratio_))
    ax_scatter_gram(ax, pca1, pca2, wordlist, title=title, x_label="第一主成分", y_label="第二主成分")

def plot_tsne(ax, R, wordlist, title=""):
    #tsne = TSNE()
    tsne_result = TSNE(n_components=2).fit_transform(R)
    print(tsne_result.shape)
    tsne1, tsne2 = tsne_result[:,0], tsne_result[:,1]
    ax_scatter_gram(ax, tsne1, tsne2, wordlist, title=title, x_label="tSNE 1", y_label="tSNE 2")


###plot_pca(ax, Onmtp_w2v, onmtp_list, title='オノマトペ附置 (PCA)')
    
fig, ax = plt.subplots(figsize=(12,13))         # Sample figsize in inches
plot_pca(ax, R_onmtp, onmtp_list, title='オノマトペ附置 (PCA)')
plt.show()

fig, ax = plt.subplots(figsize=(12,13))         # Sample figsize in inches
plot_tsne(ax, R_onmtp, onmtp_list, title='オノマトペ附置(tSNE)')
plt.show()    

In [ ]:
#東大の松下研究室の辞書を利用

import requests

url='http://www17408ui.sakura.ne.jp/tatsum/database/VDLJ_Ver1_0_General-Learners_Basic-2500.xlsx'
excel_fname = url.split('/')[-1]
r = requests.get(url)
with open(excel_fname, 'wb') as f:
    total_length = int(r.headers.get('content-length'))
    print('Downloading {0} - {1} bytes'.format(excel_fname, (total_length)))
    f.write(r.content)

x = pd.read_excel(excel_fname, sheet_name='基本語2500　Basic 2500 Words')

In [ ]:
x = pd.read_excel(excel_fname, sheet_name='基本語2500　Basic 2500 Words')
xx = x[['ふつうの（新聞の）書きかた\nStandard (Newspaper) Orthography','ふつうの読みかた（カタカナ）\nStandard Reading (Katakana)','品詞\nPart of Speech']]
x_tiny = xx.rename(columns = {'ふつうの（新聞の）書きかた\nStandard (Newspaper) Orthography':'word',
                              'ふつうの読みかた（カタカナ）\nStandard Reading (Katakana)': 'yomi',
                              '品詞\nPart of Speech':'POS'}, inplace = False)

basic_list = [w for w in list(sorted(set(x_tiny['word']))) if w in w2v]
Mat_basic = w2vMat(wordlist=basic_list)

In [ ]:
Mat_basic_norm = np.array([x/np.linalg.norm(x) for x in Mat_basic])
R_basic = Mat_basic_norm.dot(Mat_basic_norm.T)
print(R_basic.shape)

RR_df = pd.DataFrame(data=R_basic, index=basic_list)
fig, ax = plt.subplots(figsize=(12,10))         # Sample figsize in inches
sns.heatmap(RR_df, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12,13))         # Sample figsize in inches
plot_pca(ax, R_basic, basic_list, title='松下基本単語 (PCA)')
plt.show()

fig, ax = plt.subplots(figsize=(12,13))         # Sample figsize in inches
plot_tsne(ax, R_basic, basic_list, title='松下基本単語(tSNE)')
plt.show()    